In [1]:
# Import Libraries
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from tflearn.layers.normalization import local_response_normalization
import tensorflow as tf

import cv2
import numpy as np
import serial
import time 

curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Colocations handled automatically by placer.


In [2]:
# Model parameters
WIDTH = 150
HEIGHT = 150
LR = 1e-3
EPOCHS = 6

In [3]:
# labels (for images)
a = [1,0,0]
b = [0,1,0]
c = [0,0,1]

In [4]:
def one_hot_to_label(one_hot):
    if one_hot == a:
        return 'Fist'
    if one_hot == b:
        return 'OpenHand'
    if one_hot == c:
        return 'TwoFinger'

In [ ]:
def alexnet(width, height, lr):
    network = input_data(shape=[None, width, height, 1], name='input')
    network = conv_2d(network, 96, 11, strides=4, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = conv_2d(network, 256, 5, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = conv_2d(network, 384, 3, activation='relu')
    network = conv_2d(network, 384, 3, activation='relu')
    network = conv_2d(network, 256, 3, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = fully_connected(network, 4096, activation='tanh')
    network = dropout(network, 0.5)
    network = fully_connected(network, 4096, activation='tanh')
    network = dropout(network, 0.5)
    network = fully_connected(network, 3, activation='softmax')
    network = regression(network, optimizer='momentum',
                         loss='categorical_crossentropy',
                         learning_rate=lr, name='targets')

    model = tflearn.DNN(network, checkpoint_path='model_alexnet',
                        max_checkpoints=1, tensorboard_verbose=2, tensorboard_dir='log')
    return model

In [ ]:
tf.reset_default_graph()

In [ ]:
model = alexnet(WIDTH, HEIGHT, LR)

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.


In [ ]:
model.load('./model')

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from F:\GitHub\MiniRoboticArm_GestureRecognition\model_ver10


# Linking Arduino with Camera stream and model

In [ ]:
ArduinoSerial = serial.Serial('COM20', 9600)
print(ArduinoSerial.readline()) 

b'Good\r\n'


In [ ]:
def SendLabel(label):
    if (label == 0): 
        ArduinoSerial.write('0'.encode())
    if (label == 1): 
        ArduinoSerial.write('1'.encode())

In [ ]:
cap = cv2.VideoCapture(0)
prev_prediction = 0
prediction = 0
while cv2.waitKey(1):

    _, frame = cap.read()                      
    blur = cv2.GaussianBlur(frame, (5,5), 0)
    
    hsv = cv2.cvtColor(blur, cv2.COLOR_RGB2HSV)
    
    lower_skin = np.array([110, 25, 85])
    upper_skin = np.array([179, 255, 255])
    
    mask = cv2.inRange(hsv, lower_skin, upper_skin)
    blur = cv2.medianBlur(mask, 15)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (8, 8))
    hsv_d = cv2.dilate(blur, kernel)

    cv2.rectangle(frame, (0, 0), (300, 300), (255, 0, 0), 2)
    
    final = cv2.resize(hsv_d[:300, :300], (150, 150))
    one_hot = list(np.around(model.predict([final.reshape(150, 150, 1)])[0]))
    
    prev_prediction = prediction
    prediction = one_hot_to_label(one_hot)
    
    if prev_prediction != prediction:
        if prediction == 'Fist':
            SendLabel(0)
        elif prediction == 'OpenHand':
            SendLabel(1)

    cv2.putText(hsv_d,prediction, (5,25), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    cv2.imshow('Original', frame)
    cv2.imshow('Mask', hsv_d[:300, :300])
    
    if cv2.waitKey(1) & 0xFF == ord('t'):
        break
        
cv2.destroyAllWindows() 
cap.release()